In [1]:
import base64
from PIL import Image
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from vertexai.generative_models._generative_models import HarmCategory, HarmBlockThreshold, ResponseBlockedError
from pathlib import Path

vertexai.init(project="crypto-resolver-346012", location="us-central1")

def generate(image):
    model = GenerativeModel("gemini-pro-vision")
    
    safety_settings = {
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    }

    # Yes,But memes are funny because the Yes picture depicts a normal situation, and the But picture reveals something about the Yes picture that makes humans laugh. What is funny about the given Yes,But meme
    
    responses = model.generate_content(
        [image, """Describe this image"""],
        generation_config={
            "max_output_tokens": 128,
            "temperature": 0.8,
            "top_p": 1,
            "top_k": 32,
            "candidate_count": 1
        },
        safety_settings=safety_settings,
        
    )
    #print(type(responses))
    #print(responses)
    #print(responses.text)
    return responses.text


with open("images_split/20240101_172315_YES.jpg", "rb") as f:
    data = f.read()
    image1 = Part.from_data(data=data, mime_type="image/jpeg")
    #print(image1)

generate(image1)

' A pair of pink high heels with a chunky platform.'

In [3]:
import os,json
from tqdm import tqdm
from validate import validate_file

outpath = "outputs/right/gemini-pro-vision-right.json"
try:
    with open(outpath, "r") as f:
        outputs = json.load(f)
except Exception:
    print("starting from zero")
    outputs = {}

files = [x for x in os.listdir("images_split") if "BUT" in x]
redo_files = ["20240101_174949_BUT.jpg"]
for filename in tqdm(files):
    if filename in outputs and filename not in redo_files:
        continue
    with open(os.path.join("images_split",filename), "rb") as f:
        data = f.read()
        image = Part.from_data(data=data, mime_type="image/jpeg")
    #display(Image.open(os.path.join("images",filename)).convert('RGB'))
    #print(filename)
    try:
        output = generate(image)
    except Exception as e:
        print("Caught exception: ",str(e))
        print("Could not do file: ",filename)
        output = ""
    
    outputs[filename]= output
    with open(outpath, "w") as f:
        json.dump(outputs, f, indent=4)

with open(outpath, "w") as f:
    json.dump(outputs, f, indent=4)
if not validate_file(outpath):
    print("Validation failed!")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 283/283 [00:06<00:00, 41.31it/s]

File outputs/right/gemini-pro-vision-right.json is valid.


In [ ]:
"""
Caught exception:  500 Internal error encountered.
Could not do file:  20240101_172932.jpg

Caught exception:  500 Internal error encountered.
Could not do file:  20240101_182249.jpg
"""

In [ ]:
"""
manually redid:
["20240101_173205_BUT.jpg", "20240101_173205_YES.jpg", "20240101_174949_BUT.jpg", "20240101_174949_YES.jpg", "20240101_181310_YES.jpg"]
"""